In [1]:
%matplotlib inline
import isce
import os
import glob
import numpy as np
import shutil
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
from uavsar_pytools.georeference import geolocate_uavsar
from pathlib import Path
from glob import glob
from uavsar_pytools.convert.tiff_conversion import read_annotation, array_to_tiff
from os.path import join, basename, dirname
from rasterio.vrt import WarpedVRT
from osgeo import gdal, osr

2024-03-15 12:02:14,443 - rasterio.session - DEBUG - Could not import boto3, continuing with reduced functionality.


/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
### still need to hand format unw vrt files....

In [14]:
# combine lkv files
def combo_lkvs(data_dir: Path, ann_fp: Path):
    """
    Combines segment .lkv files into a single combined .lkv file and saves as a binary with .vrt.
    """
    assert data_dir.exists()

    re_lkvs = {'east':[], 'north': [], 'up':[]}
    for lkvs in sorted(data_dir.glob('*.lkv')):
        segment = lkvs.stem.split('_')[-2].replace('s','')

        data = np.fromfile(lkvs, np.dtype('<f'))
        east, north, up = data[::3], data[1::3], data[2::3]
        for key, da in zip(re_lkvs.keys(), [east, north, up]):
            re_lkvs[key].extend(da)
                               
    full = np.empty(len(re_lkvs['east'])*3, dtype='>f')
    full[0::3] = re_lkvs['east']
    full[1::3] = re_lkvs['north']
    full[2::3] = re_lkvs['up']

    # read ann file
    desc = read_annotation(ann_fp)
    print('reading .ann file')

    # read in number of rows from each lvk file
    nrows1 = desc[f'lkv_1_2x8.set_rows']['value']
    nrows2 = desc[f'lkv_2_2x8.set_rows']['value']

    # add rows for new reshaping number
    nrows_new = nrows1 + nrows2
    print('new number of rows = ',nrows_new)
    
    # read in cols, same fo rboth
    ncols = desc[f'lkv_1_2x8.set_cols']['value']
    dt = np.dtype('<f')

    # create empty arrays for lat and lon
    east_array = np.empty((nrows_new, ncols))
    north_array = np.empty((nrows_new, ncols))
    up_array = np.empty((nrows_new, ncols))
    
    
    # fill each layer
    east_array = full[::3].reshape(nrows_new, ncols)
    north_array = full[1::3].reshape(nrows_new, ncols)
    up_array = full[2::3].reshape(nrows_new, ncols)

    # define path to bin file
    east_output_file = data_dir / "multi_seg.east"
    north_output_file = data_dir / "multi_seg.north"
    up_output_file = data_dir / "multi_seg.up"


    #### Save the array to bin file
    # east
    east_array.tofile(east_output_file)
    print('.east saved')
    
    # north
    north_array.tofile(north_output_file)
    print('.north saved')

    # up
    up_array.tofile(up_output_file)
    print('.up saved')

    profile = {
    'driver': 'GTiff',
    'interleave': 'band',
    'tiled': False,
    'nodata': 0,
    'width': ncols,
    'height':nrows_new,
    'count':1,
    'dtype':'float32'
    }
    
    ### Save out tifs to be converted to vrt
    # east
    with rio.open(join(str(east_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(east_array.astype(east_array.dtype), 1)
        
    # north
    with rio.open(join(str(north_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(north_array.astype(north_array.dtype), 1)
    # north
    with rio.open(join(str(up_output_file) + '.tif'), 'w', **profile) as dst:
                dst.write(up_array.astype(up_array.dtype), 1)

    # Add VRT file for each tif
    tifs = glob(join(data_dir, '*.tif')) # list all .lvk files
    for tiff in tifs: # loop to open and translate .lvk to .vrt, and save .vrt using gdal
        raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
        raster = gdal.Translate(join(data_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float32)
    raster_dataset = None

    print('new .east, .north, and .up with .vrt have saved')

In [15]:
# start in downloads directory
os.chdir('/Users/jtarrico/ch3_fusion/rasters/')

In [16]:
dir1 = Path('./cop_dem_30m/')
ann1 = Path('./cop_dem_30m/sierra_17305_20002_001_200131_L090HH_01_BC.ann')

In [17]:
full1 = combo_lkvs(data_dir = dir1, ann_fp = ann1)

reading .ann file
new number of rows =  16212
.east saved
.north saved
.up saved
2024-03-15 12:16:31,481 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x1a6c34700>
2024-03-15 12:16:31,483 - rasterio.env - DEBUG - Starting outermost env
2024-03-15 12:16:31,484 - rasterio.env - DEBUG - No GDAL environment exists
2024-03-15 12:16:31,485 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x1a6fc3c40> created
2024-03-15 12:16:31,488 - rasterio._filepath - DEBUG - Installing FilePath filesystem handler plugin...
2024-03-15 12:16:31,489 - rasterio._env - DEBUG - GDAL_DATA found in environment.
2024-03-15 12:16:31,490 - rasterio._env - DEBUG - PROJ_DATA found in environment.
2024-03-15 12:16:31,492 - rasterio._env - DEBUG - Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x1a6fc3c40>.
2024-03-15 12:16:31,493 - rasterio.env - DEBUG - Entered env context: <rasterio.env.Env object at 0x1a6c34700>
2024-03-15 12:16:31,495 - rasterio._i

/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


2024-03-15 12:16:32,742 - rasterio.env - DEBUG - Exiting env context: <rasterio.env.Env object at 0x19ef42430>
2024-03-15 12:16:32,743 - rasterio.env - DEBUG - Cleared existing <rasterio._env.GDALEnv object at 0x1a6e8c7c0> options
2024-03-15 12:16:32,745 - rasterio._env - DEBUG - Stopped GDALEnv <rasterio._env.GDALEnv object at 0x1a6e8c7c0>.
2024-03-15 12:16:32,746 - rasterio.env - DEBUG - Exiting outermost env
2024-03-15 12:16:32,747 - rasterio.env - DEBUG - Exited env context: <rasterio.env.Env object at 0x19ef42430>
2024-03-15 12:16:32,748 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x1a47ad100>
2024-03-15 12:16:32,750 - rasterio.env - DEBUG - Starting outermost env
2024-03-15 12:16:32,750 - rasterio.env - DEBUG - No GDAL environment exists
2024-03-15 12:16:32,752 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x1a6c34700> created
2024-03-15 12:16:32,753 - rasterio._env - DEBUG - GDAL_DATA found in environment.
2024-03-1

# p1

In [34]:
# path to your unw.vrt
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.vrt'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_unw.tif'

In [35]:
# p1 unw
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


In [13]:
# p1 concomp
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_conncomp.tif'

In [14]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...


Warning 1: for band 1, destination nodata value has been clamped to 0, the original value being out of range.


done!


In [15]:
# p1 coh
in_fp = 'int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.coh'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p1/p1_14d_VV_coh.tif'

In [16]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


# p2

In [17]:
# p2 unw
in_fp = 'int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw.vrt'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p2/p2_7d_VV_unw.tif'

In [18]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


In [19]:
# p2 concomp
in_fp = 'int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p2/p2_7d_VV_conncomp.tif'

In [20]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...


Warning 1: for band 1, destination nodata value has been clamped to 0, the original value being out of range.


done!


In [21]:
# p2 coh
in_fp = 'int/20200212T2211_20200219T2208/20200212T2211_20200219T2208.coh'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p2/p2_7d_VV_coh.tif'

In [22]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


# p3

In [23]:
# p3 unw
in_fp = 'int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.unw_snaphu.unw.vrt'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p3/p3_7d_VV_unw.tif'

In [24]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


In [28]:
# p3 conncomp
in_fp = 'int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p3/p3_7d_VV_conncomp.tif'

In [ ]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

In [29]:
# p3 coh
in_fp = 'int/20200219T2208_20200226T2253/20200219T2208_20200226T2253.coh'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p3/p3_7d_VV_coh.tif'

In [30]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


# p4

In [31]:
# p4 unw
in_fp = 'int/20200226T2253_20200311T1852/20200226T2253_20200311T1852.unw_snaphu.unw.vrt'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p4/p4_14d_VV_unw.tif'

In [32]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!


In [ ]:
# p4 conncomp
in_fp = 'int/20200226T2253_20200311T1852/20200226T2253_20200311T1852.'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p4/p4_14d_VV_conncomp.tif'

In [ ]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

In [33]:
# p4 conncomp
in_fp = 'int/20200226T2253_20200311T1852/20200226T2253_20200311T1852.unw_snaphu.unw.conncomp'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p4/p4_14d_VV_conncomp.tif'

In [34]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...


Warning 1: for band 1, destination nodata value has been clamped to 0, the original value being out of range.


done!


In [35]:
# p4 conncomp
in_fp = 'int/20200226T2253_20200311T1852/20200226T2253_20200311T1852.coh'

# path to annotation file
lat_fp = 'int/new_llh/multi_seg.lat.vrt'

# path to annotation file
lon_fp = 'int/new_llh/multi_seg.lon.vrt'

# where the .tif is being save
out_fp = '/Users/jtarrico/sierra_isce_multi/insar_geocoded/p4/p4_14d_VV_coh.tif'

In [36]:
geocodeUsingGdalWarp(infile = in_fp,
                     latfile = lat_fp, 
                     lonfile = lon_fp, 
                     outfile = out_fp,
                     insrs=4326, outsrs=None,spacing=[.00005556,.00005556], fmt='GTiff', bounds=None,method='near')

geocoding...
done!
